# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to damien.pretet@me.com and will expire on September 17, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1477930234.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.748490     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.810204     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
model_all.get("coefficients")

name,index,value,stderr
(intercept),None,274873.05595,None
bedrooms,None,0.0,None
bedrooms_square,None,0.0,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

Answers: bathrooms, sqft_living, sqft_living_sqrt

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors'] = testing['floors'].astype(float) 
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors'] = training['floors'].astype(float) 
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors'] = validation['floors'].astype(float) 
validation['floors_square'] = validation['floors']*validation['floors']

In [10]:
import numpy as np

In [11]:
def get_residual_sum_of_squares(model, data, outcome):
    residuals = 0
    RSS = 0
    # First get the predictions
    predictions = model.predict(data)
    
    i=0
    while i < outcome.size():
        # Then compute the residuals/errors
        residuals = outcome[i]  - predictions[i]
        # Then square and add them up
        RSS +=  residuals * residuals
        i += 1 

    return(RSS) 

In [15]:
models = []
lower = 0
better = 0
i = 0

for penalty in np.logspace(1, 7, num=13):

    print "Training model with l1_penalty="  + str(penalty)
    
    model = {}
    model["l1_penalty"] = penalty
    model["model"] = graphlab.linear_regression.create(training, 
                                                       target='price',
                                                       features=all_features,
                                                       validation_set=None, 
                                                       l2_penalty=0., 
                                                       l1_penalty=penalty,
                                                       verbose = False)
    
    model["RSS"] = get_residual_sum_of_squares(model["model"], validation, validation["price"])
    
    # Initialize lower with first value
    if i == 1:
        lower = model["RSS"]
    elif model["RSS"] < lower:
        print "model %d with penalty %f becomes the better model" % i , penalty
        lower = model["RSS"]
        better = i
    
    models.append(model)
    print ""
    
    i += 1

print "Best model uses l1_penalty = %f " % models[better]["l1_penalty"]

Training model with l1_penalty=10.0

Training model with l1_penalty=31.6227766017

Training model with l1_penalty=100.0

Training model with l1_penalty=316.227766017

Training model with l1_penalty=1000.0

Training model with l1_penalty=3162.27766017

Training model with l1_penalty=10000.0

Training model with l1_penalty=31622.7766017

Training model with l1_penalty=100000.0

Training model with l1_penalty=316227.766017

Training model with l1_penalty=1000000.0

Training model with l1_penalty=3162277.66017

Training model with l1_penalty=10000000.0

Best model uses l1_penalty = 10.000000 


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [16]:
i = 0
lower = 0
while i < len(models):
    if models[i]["RSS"] < models[lower]["RSS"]:
        lower = i
        print "l1_penalty = " + str(model["RSS"]) + " is lower"
    i += 1
print "Lowest RSS is for l1-penalty=%f (index=%d)" % (models[lower]["l1_penalty"], lower)

Lowest RSS is for l1-penalty=10.000000 (index=0)


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [17]:
models[lower]["model"].get("coefficients").print_rows(num_rows=len(models[lower]["model"].get("coefficients")))

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [18]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [19]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [20]:
l1_penalties = []
for l1_penalty in l1_penalty_values:
    
    print "Training model with l1_penalty="  + str(l1_penalty)
    
    model = {}
    model["l1_penalty"] = l1_penalty
    model["model"] = graphlab.linear_regression.create(training, 
                                                       target='price', 
                                                       features=all_features,
                                                       validation_set=None, 
                                                       l2_penalty=0., l1_penalty=l1_penalty,
                                                       verbose = False)
    
    if model["model"]['coefficients']["value"].nnz() <= max_nonzeros:
        l1_penalties.append(l1_penalty)
    print model["model"]['coefficients']["value"].nnz()
    
print l1_penalties

Training model with l1_penalty=100000000.0
18
Training model with l1_penalty=127427498.57
18
Training model with l1_penalty=162377673.919
18
Training model with l1_penalty=206913808.111
18
Training model with l1_penalty=263665089.873
17
Training model with l1_penalty=335981828.628
17
Training model with l1_penalty=428133239.872
17
Training model with l1_penalty=545559478.117
17
Training model with l1_penalty=695192796.178
17
Training model with l1_penalty=885866790.41
16
Training model with l1_penalty=1128837891.68
15
Training model with l1_penalty=1438449888.29
15
Training model with l1_penalty=1832980710.83
13
Training model with l1_penalty=2335721469.09
12
Training model with l1_penalty=2976351441.63
10
Training model with l1_penalty=3792690190.73
6
Training model with l1_penalty=4832930238.57
5
Training model with l1_penalty=6158482110.66
3
Training model with l1_penalty=7847599703.51
1
Training model with l1_penalty=10000000000.0
1
[3792690190.7322536, 4832930238.5717525, 61584821

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [21]:
l1_penalty_min = 2976351441
l1_penalty_max = 3792690190

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [22]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [31]:
models = []
best = 0
lower = -1

for l1_penalty in l1_penalty_values:
    
    print "Training model with l1_penalty="  + str(l1_penalty)
    
    model = {}
    model["l1_penalty"] = l1_penalty
    model["model"] = graphlab.linear_regression.create(training, 
                                                       target='price', 
                                                       features=all_features,
                                                       validation_set=None, 
                                                       l2_penalty=0., l1_penalty=l1_penalty,
                                                       verbose = False)
    
    model["RSS"] = get_residual_sum_of_squares(model["model"], validation, validation["price"])
    print "RSS = " + str(model["RSS"])
    model["nnz"] = model["model"]['coefficients']["value"].nnz()
    
    if model["nnz"] == 7 and lower == -1:
        lower = model
    
    if model["nnz"] == 7:
        if model["RSS"] < lower["RSS"]:
            print "Just found a better model!"
            print l1_penalty
    
    print "Number of none zero coeffs: %d " % model["model"]['coefficients']["value"].nnz()
    models.append(model)
    print ""
    
print "Best model is:"
print "    RSS=%f" % lower["RSS"]
print "    l1_penalty=%f" % lower["l1_penalty"]

Training model with l1_penalty=2976351441.0
RSS = 9.66925692362e+14
Number of none zero coeffs: 10 

Training model with l1_penalty=3019316638.32
RSS = 9.74019450085e+14
Number of none zero coeffs: 10 

Training model with l1_penalty=3062281835.63
RSS = 9.81188367942e+14
Number of none zero coeffs: 10 

Training model with l1_penalty=3105247032.95
RSS = 9.89328342459e+14
Number of none zero coeffs: 10 

Training model with l1_penalty=3148212230.26
RSS = 9.98783211266e+14
Number of none zero coeffs: 10 

Training model with l1_penalty=3191177427.58
RSS = 1.00847716702e+15
Number of none zero coeffs: 10 

Training model with l1_penalty=3234142624.89
RSS = 1.01829878055e+15
Number of none zero coeffs: 10 

Training model with l1_penalty=3277107822.21
RSS = 1.02824799221e+15
Number of none zero coeffs: 10 

Training model with l1_penalty=3320073019.53
RSS = 1.03461690923e+15
Number of none zero coeffs: 8 

Training model with l1_penalty=3363038216.84
RSS = 1.03855473594e+15
Number of none 

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [27]:
i = 0
lower = 0
while i < len(models):
    if models[i]["RSS"] < models[lower]["RSS"] and models[i]["nnz"] == max_nonzeros:
        lower = i
        print "l1_penalty = " + str(model["RSS"]) + " is lower"
    i += 1
print "Lowest RSS is for l1_penalty=%f (index=%d)" % (models[lower]["l1_penalty"], lower)

Lowest RSS is for l1_penalty=2976351441.000000 (index=0)


In [25]:
models[lower]["model"].get("coefficients")

name,index,value,stderr
(intercept),None,196100.937806,None
bedrooms,None,2181.57432107,None
bedrooms_square,None,0.0,None
bathrooms,None,17962.6966612,None
sqft_living,None,34.1424656512,None
sqft_living_sqrt,None,789.319789078,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,3665.9308176,None
floors_square,None,0.0,None


In [26]:
models[lower]["model"].get("coefficients").print_rows(num_rows=len(models[lower]["model"].get("coefficients")))

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 196100.937806 |  None  |
|     bedrooms     |  None | 2181.57432107 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 17962.6966612 |  None  |
|   sqft_living    |  None | 34.1424656512 |  None  |
| sqft_living_sqrt |  None | 789.319789078 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |  3665.9308176 |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None | 11333.8410308 |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 3578.90040044 |  None  |
|    sqft_above    |  None | 32.7432013718 |  None  |
|  sqft_basement   |  None |

In [32]:
lower["model"].get("coefficients").print_rows(num_rows=len(lower["model"].get("coefficients")))

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |